In [3]:
import pandas as pd
import spacy
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

nlp = spacy.load("en_core_web_lg")

In [4]:
business_name = "Shake Shack"
reviews = pd.read_pickle("data/" + business_name + "_spacy")

In [21]:
search_query = "location"
search_doc = nlp(search_query)

def calc_similarity(review_doc):
    return review_doc.similarity(search_doc)

reviews['similarity'] = reviews.spacy.progress_apply(calc_similarity)
reviews.sort_values('similarity', ascending=False, inplace=True)

In [27]:
from IPython.display import HTML, display
import tabulate

In [ ]:
# NOUN, VERB, ADJ (CCONJ ADJ VERB)
# ADJ NOUN/PROPN
# ADV VERB
# ADJ ADJ NOUN/PROPN
# ADV VERB
# PROPN VERB ADJ
# PROPN PROPN PROPN

In [41]:
accepted_pos = {'NOUN', 'PROPN', 'ADV', 'ADJ', 'VERB'}
after_cconj_pos = {'ADV', 'ADJ', 'VERB'}
permitted_start_ends = {('NOUN', 'ADJ'), ('NOUN', 'VERB'), ('PROPN', 'ADJ'), 
                        ('PROPN', 'VERB'), ('ADJ', 'NOUN'), ('ADJ', 'PROPN'), 
                        ('ADV', 'VERB'), ('PROPN', 'NOUN'), ('NOUN', 'PROPN'), 
                        ('NOUN', 'NOUN'), ('PROPN', 'PROPN')}

In [ ]:
def pull_phrases(doc):

In [64]:
i = 2
doc = reviews.iloc[i]['spacy']
phrases = []

start_index = 0 # should be inclusive
end_index = 0 # should be exclusive

while start_index < len(doc):
    while start_index < len(doc) and doc[start_index].pos_ not in accepted_pos:
        start_index+=1
    end_index = start_index
    while end_index < len(doc) and doc[end_index].pos_ in accepted_pos:
        end_index += 1
        if end_index < len(doc) - 1 and doc[end_index].pos_ == 'CCONJ' and doc[end_index+1].pos_ in after_cconj_pos:
            end_index += 2 # include related terms after conjunctions
        
    # add phrase
    phrase = []
    for token_index in range(start_index, end_index):
        phrase.append(doc[token_index].lemma_)
    phrase = ' '.join(phrase)
    if len(phrase) > 1:
        phrases.append(phrase)
    
    # Move start_index to end_index, to continue scanning
    start_index = end_index
print(doc)
print(phrases)

Still the same great taste, unfortunately the customer service is lacking compared to the summerlin location.
['still', 'same great taste', 'unfortunately', 'customer service be lack compare', 'summerlin location']


In [48]:
phrases

['delicious french fries',
 'yummy hot dog',
 'atmosphere be very casual but pleasant',
 'very convenient location',
 'downtown summerlin']

In [63]:
for i in range(1, 8):
    display(reviews.iloc[i]['text'])
    doc = reviews.iloc[i]['spacy']
    output = []
    for token in doc:
        output.append([token.text, token.lemma_, token.pos_])
    display(HTML(tabulate.tabulate(output, tablefmt='html')))

'So happy to have this location in Charlotte. .. centrally located in the Park Road Shopping Center. Just as good as the ones I remember from out of state'

So,so,ADV
happy,happy,ADJ
to,to,PART
have,have,VERB
this,this,DET
location,location,NOUN
in,in,ADP
Charlotte,charlotte,PROPN
.,.,PUNCT
..,..,PUNCT
centrally,centrally,ADV


'Still the same great taste, unfortunately the customer service is lacking compared to the summerlin location.'

Still,still,ADV
the,the,DET
same,same,ADJ
great,great,ADJ
taste,taste,NOUN
",",",",PUNCT
unfortunately,unfortunately,ADV
the,the,DET
customer,customer,NOUN
service,service,NOUN
is,be,VERB


"This location of Shake Shack is the one to visit in AZ.  Consistently good burgers and good service.  Has not yet been a disappointing experience at the Central location.  Props to this location keep it up and please don't change."

...

Food,food,NOUN
:,:,PUNCT
5/5,5/5,NUM
,,
Service,service,PROPN
:,:,PUNCT
5/5,5/5,NUM
,,
Cleanliness,cleanliness,PROPN
:,:,PUNCT
5/5,5/5,NUM


'Delicious French Fries, yummy Hot Dog and the atmosphere was very casual but pleasant.  Very convenient location in Downtown Summerlin.'

Delicious,delicious,PROPN
French,french,PROPN
Fries,fries,PROPN
",",",",PUNCT
yummy,yummy,ADJ
Hot,hot,PROPN
Dog,dog,PROPN
and,and,CCONJ
the,the,DET
atmosphere,atmosphere,NOUN
was,be,VERB


'Amazing burgers, great fries, quick and simple place to get a burger. Very good service, the decor is cool and location is good, right outside of New York New York hotel.'

Amazing,amazing,ADJ
burgers,burger,NOUN
",",",",PUNCT
great,great,ADJ
fries,fry,NOUN
",",",",PUNCT
quick,quick,ADJ
and,and,CCONJ
simple,simple,ADJ
place,place,NOUN
to,to,PART
